In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from numpy import genfromtxt
from pandas_profiling import ProfileReport

In [2]:
df = pd.read_csv("/workspaces/SigmoidFreud/Data/X_test_window_size_15_time_encoding_True.csv", sep=',')
profile = ProfileReport(df, title="Pandas Profiling Report")
profile.to_file("/workspaces/SigmoidFreud/Data/X_window_size_15.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3496 entries, 0 to 3495
Columns: 101 entries, total_hours to 83
dtypes: float64(101)
memory usage: 2.7 MB


In [4]:
df.head(5)

,total_hours,dlvry_weekend,dlvry_bank_holiday,dlvry_day_sin,dlvry_day_cos,dlvry_weekday_sin,dlvry_weekday_cos,dlvry_hour_sin,dlvry_hour_cos,lasttrade_weekend,...,74,75,76,77,78,79,80,81,82,83
0,22.999652,0.0,0.0,0.998717,-0.050649,0.433884,-0.900969,1.224647e-16,-1.000000,0.0,...,0.36,5.00,43.3,300.0,4.97,7.81,2.13,5.57,90.1,240.0
1,22.999652,0.0,0.0,0.998717,-0.050649,0.433884,-0.900969,1.224647e-16,-1.000000,0.0,...,2.13,5.57,90.1,240.0,6.87,2.30,3.67,4.44,433.4,180.0
2,23.999380,0.0,0.0,0.998717,-0.050649,0.433884,-0.900969,-2.588190e-01,-0.965926,0.0,...,-0.10,-2.25,7.2,660.0,-1.00,-1.28,-3.06,-2.50,-4.9,600.0
3,23.999380,0.0,0.0,0.998717,-0.050649,0.433884,-0.900969,-2.588190e-01,-0.965926,0.0,...,-3.06,-2.50,-4.9,600.0,-3.07,-1.38,-6.13,-6.69,73.0,540.0
4,23.999380,0.0,0.0,0.998717,-0.050649,0.433884,-0.900969,-2.588190e-01,-0.965926,0.0,...,-6.13,-6.69,73.0,540.0,-5.18,1.71,-1.21,0.80,318.6,480.0


In [5]:
df.shape

(3496, 101)

In [6]:
X_train = genfromtxt("/workspaces/SigmoidFreud/Data/X_train_window_size_15_time_encoding_True.csv", delimiter=',', skip_header=1)
X_valid = genfromtxt("/workspaces/SigmoidFreud/Data/X_valid_window_size_15_time_encoding_True.csv", delimiter=',', skip_header=1)
X_test = genfromtxt("/workspaces/SigmoidFreud/Data/X_test_window_size_15_time_encoding_True.csv", delimiter=',', skip_header=1)

y_train = genfromtxt("/workspaces/SigmoidFreud/Data/y_train_window_size_15_time_encoding_True.csv", delimiter=',', skip_header=1)
y_valid = genfromtxt("/workspaces/SigmoidFreud/Data/y_valid_window_size_15_time_encoding_True.csv", delimiter=',', skip_header=1)
y_test = genfromtxt("/workspaces/SigmoidFreud/Data/y_test_window_size_15_time_encoding_True.csv", delimiter=',', skip_header=1)


In [7]:
from sklearn.metrics import mean_squared_error 
from math import sqrt
def evaluate_model(model, X_valid, y_valid_true):
    predictions = model.predict(X_valid)
    rms = sqrt(mean_squared_error(y_valid_true, predictions))
    print("Root mean squared error on valid:",rms)
    return rms

In [8]:
dmatrix_train = xgb.DMatrix(data=X_train,label=y_train) 
dmatrix_valid = xgb.DMatrix(data=X_valid,label=y_valid)
dmatrix_test = xgb.DMatrix(data=X_test,label=y_test)

In [9]:
params = {'objective': 'reg:linear', 'eval_metric': 'rmse', 'n_estimators': 100000}
evallist = [(dmatrix_valid, 'eval'), (dmatrix_train, 'train')]
xg_reg = xgb.train(params = params, dtrain=dmatrix_train, num_boost_round=50, evals=evallist)

[15:33:29] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:33:29] WARNING: ../src/learner.cc:573: 
Parameters: { "n_estimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	eval-rmse:7.20681	train-rmse:4.20865
[1]	eval-rmse:7.26896	train-rmse:4.09787
[2]	eval-rmse:7.30107	train-rmse:3.97499
[3]	eval-rmse:7.30736	train-rmse:3.91461
[4]	eval-rmse:7.39444	train-rmse:3.82837
[5]	eval-rmse:7.38107	train-rmse:3.80249
[6]	eval-rmse:7.36258	train-rmse:3.75676
[7]	eval-rmse:7.38545	train-rmse:3.73744
[8]	eval-rmse:7.39107	train-rmse:3.70827
[9]	eval-rmse:7.38613	train-rmse:3.63538
[10]	eval-rmse:7.44490	train-rmse:3.54707
[11]	eval-rmse:7.53555	train-rmse:3.49888
[12]	eval-rmse:7.54162	train-rmse:3.46026
[13]	eval-r

In [10]:
result = evaluate_model(xg_reg, dmatrix_test, dmatrix_test.get_label())

Root mean squared error on valid: 6.358427459833358


In [11]:
from sklearn.ensemble import RandomForestRegressor
RF_base_model = RandomForestRegressor(n_estimators = 10, random_state=42069, n_jobs= -1)
RF_base_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=42069)

In [12]:
result = evaluate_model(RF_base_model, X_test, y_test)

Root mean squared error on valid: 6.782831154885582
